In [7]:
from src.data_loader import MindDataset
from transformers import AutoTokenizer
import pandas as pd

In [3]:
tokenizer = AutoTokenizer.from_pretrained('bert-base-uncased')

c:\Users\Mark\anaconda3\envs\research\Lib\site-packages\huggingface_hub\file_download.py:133: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Mark\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [4]:
train_set = MindDataset(
    root='data',
    tokenizer=tokenizer,
    mode='train',
    split='small',
    news_max_len=20,
    hist_max_len=20,
    seq_max_len=300
)

Saving news info from data\small\news_dict.pkl


In [8]:
df = pd.read_csv('./data/large/train/news.tsv', sep='\t', header=None, 
                    names=['news_id', 'category', 'sub-category', 'title', 'abstract', 'url', 'title entities', 'abstract entities'])

In [9]:
df_dev = pd.read_csv('./data/large/dev/news.tsv', sep='\t', header=None, 
                    names=['news_id', 'category', 'sub-category', 'title', 'abstract', 'url', 'title entities', 'abstract entities'])
df_test = pd.read_csv('./data/large/test/news.tsv', sep='\t', header=None, 
                    names=['news_id', 'category', 'sub-category', 'title', 'abstract', 'url', 'title entities', 'abstract entities'])

In [10]:
df_all = pd.concat([df, df_dev, df_test])

In [12]:
categories = df_all['category'].unique()

In [13]:
categories

array(['lifestyle', 'news', 'health', 'sports', 'weather',
       'entertainment', 'foodanddrink', 'autos', 'travel', 'video',
       'finance', 'tv', 'movies', 'music', 'kids', 'middleeast', 'games',
       'northamerica'], dtype=object)

In [15]:
sub_categories = df_all['sub-category'].unique()

In [16]:
sub_categories

array(['lifestyleroyals', 'newsscienceandtechnology', 'weightloss',
       'medical', 'newsworld', 'voices', 'cardio', 'football_nfl',
       'weathertopstories', 'gaming', 'recipes', 'lifestylelovesex',
       'nutrition', 'autosenthusiasts', 'autossports', 'wellness',
       'health-news', 'celebrity', 'travelarticle', 'autossuvs',
       'newspolitics', 'more_sports', 'traveltripideas', 'animals',
       'autosnews', 'newsbusiness', 'golf', 'newstrends',
       'lifestylepetsanimals', 'finance-insurance', 'football_ncaa',
       'lifestylebuzz', 'mma', 'fitness', 'newsus', 'tv-gallery',
       'tvnews', 'lifestylehoroscope', 'basketball_nba', 'news',
       'shop-all', 'newsphotos', 'lifestylemindandsoul', 'travelnews',
       'basketball_ncaa', 'finance-real-estate', 'quickandeasy',
       'tv-celebrity', 'financenews', 'lifestyleparenting',
       'movies-gallery', 'racing', 'tipsandtricks', 'baseball_mlb',
       'musicnews', 'autosbuying', 'shop-apparel', 'autostrucks',
       '